# sprint21 RNN

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
import math

pd.set_option('display.max_columns', 250)
np.set_printoptions(threshold=np.inf)
import tensorflow as tf

In [3]:
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 1, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

## 【問題1】SimpleRNNのフォワードプロパゲーション実装  
## 【問題2】小さな配列でのフォワードプロパゲーションの実験

In [10]:
class Tanh:
    def __init__(self):
        self.out = None
    def forward(self,x):
#          out = 1/(1 + np.exp(-x))
        out = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        self.out = out
        return out
    def backward(self,dout):
        dx = dout * (1 - self.out**2) ### 確認。out?
#         dx = dout * (1.0 - self.out) * self.out
        return dx

In [7]:
class SGD:

    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        layer.W = layer.W - self.lr * layer.dW
        layer.B = layer.B - self.lr * layer.dB.mean(axis = 0) 


In [6]:
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
    def WI(self, n_nodes1, n_nodes2):
        WI = np.random.randn(n_nodes1, n_nodes2) * self.sigma
        return WI
    def BI(self, n_nodes2):
        BI = np.random.randn(n_nodes2) * self.sigma
        return BI

In [18]:
class RNN_layer:

    def __init__(self,batch_size, n_sequence, n_features, initializer, optimizer):
        self.optimizer = optimizer
        self.initializer = initializer
        
        self.batch_size = batch_size
        self.n_sequence = n_sequence
        self.n_nodes = 4
        self.n_features = n_features

        self.Wx = np.zeros([self.n_features, self.n_nodes])
        self.Wh = np.zeros([self.n_nodes, self.n_nodes])
        self.B =  np.zeros(self.n_nodes)
        
    def forward(self, Z, H):
        self.Z = Z
        self.H = H
        self.activation1=Tanh()

        for i in range(self.batch_size):
            for j in range(self.n_sequence):
                A = np.dot(self.Z[i][j], self.Wx) + np.dot(self.H, self.Wh)+ self.B
                self.H = self.activation1.forward(A)
        return self.H
  
#     def backward(self, dA):
#         pass

## Ｄｉｖｅｒの値と一致した。

In [19]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)

R1 = RNN_layer(1,3,2,SimpleInitializer,SGD)

R1.Wx = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
R1.Wh= np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
R1.B = np.array([1,1,1,1])

R1.forward(x, np.array([0,0,0,0]))

array([0.79494228, 0.81839002, 0.83939649, 0.85584174])

## 【問題3】（アドバンス課題）バックプロパゲーションの実装（省略）